In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from dateutil.parser import parse
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPRegressor
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import ADASYN
from collections import Counter
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier

from sklearn.linear_model import LogisticRegression


In [76]:
janco_sub = pd.read_csv( F'./results/janco_submission.csv').rename(columns={'TX_FRAUD':'TX_F_J'})
results_5_xgb = pd.read_csv( F'./results/results_5_XGB.csv').rename(columns={'TX_FRAUD':'TX_F_XGB'})
results_6_xgb = pd.read_csv( F'./results/results_6_XGB.csv').rename(columns={'TX_FRAUD':'TX_F_XGB_6'})
results_5_Forest = pd.read_csv( F'./results/results_5_forest.csv').rename(columns={'TX_FRAUD':'TX_F_Forest'})



In [46]:
janco_sub.count()

TX_ID     71139
TX_F_J    71139
dtype: int64

In [77]:
merged_sub = pd.merge(janco_sub,results_5_xgb,on = 'TX_ID')
merged_sub = pd.merge(merged_sub,results_5_Forest,on = 'TX_ID')
merged_sub = pd.merge(merged_sub,results_6_xgb,on = 'TX_ID')


In [48]:
merged_sub.count()

TX_ID       71139
TX_F_J      71139
TX_F_XGB    71139
TX_F_BC     71139
dtype: int64

In [52]:
merged_sub.head(10)

,TX_ID,TX_F_J,TX_F_XGB,TX_F_Forest
0,09324d812ba7915c3f791e973db293ad50db70d8,0.006613,0.007558,0.015625
1,a17bc798bacadbde6cd7928688e2453fa6023d67,0.000395,0.007480,0.000000
2,dee3efed3600425f60eee0e9d363f9bac226e12a,0.016188,0.012160,0.003906
3,e0b665434d7945d6b170eb8da1b88d3fdfaaf3f7,0.006988,0.004564,0.011719
4,b25981de8b5dcdc1c38de111a21312115c34199d,0.005602,0.003698,0.003906
5,2e1a61a47232ce57023aeb653ffb9146ee36a819,0.041178,0.006710,0.007812
6,418be3d6fef8443de6c8caa171ac82e01784761c,0.001783,0.005500,0.000000
7,4d64064dad4c900e4d0dbaf4b327520b1bbe8a4c,0.011337,0.006000,0.011719
8,95486443ebebb454403ec8018c555c15fb0b5fbd,0.019377,0.015780,0.003906
9,75dc228b5a044c8588a9cf8039273c1a41a73e56,0.003787,0.063808,0.121094


In [78]:
def getVote(pred1,pred2,pred3,pred4):
	if (pred1 < 0.1 and pred2 < 0.1 and pred3 < 0.1 and pred4 < 0.1):
		return 0
	elif (pred1 > 0.9 and pred2 > 0.9 and pred3 > 0.9 and pred4 > 0.9):
		return 1
	else:
		sum = pred1 +pred2 +pred3+pred4
		return sum/4

In [79]:
merged_sub['merged'] = merged_sub.apply(lambda row: getVote(row.TX_F_J,row.TX_F_XGB,row.TX_F_XGB_6,row.TX_F_Forest),axis=1)

In [80]:
test_trans = pd.read_csv( F'./data/transactions_test.csv')

In [81]:
fraud_by_amount = test_trans[test_trans['TX_AMOUNT'] > 142]

In [82]:
fraud_by_cashback = test_trans[test_trans['TRANSACTION_CASHBACK_AMOUNT'] > 25]

In [83]:
merged_sub.loc[fraud_by_amount.index,'merged']= 1

In [84]:
submission = merged_sub[['TX_ID','merged']]

In [85]:
submission.rename(columns={'merged':'TX_FRAUD'},inplace=True)


C:\Users\joshu\AppData\Local\Temp\ipykernel_26588\2188786321.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission.rename(columns={'merged':'TX_FRAUD'},inplace=True)


In [86]:
submission.head(30)

,TX_ID,TX_FRAUD
0,09324d812ba7915c3f791e973db293ad50db70d8,0.000000
1,a17bc798bacadbde6cd7928688e2453fa6023d67,0.000000
2,dee3efed3600425f60eee0e9d363f9bac226e12a,0.000000
3,e0b665434d7945d6b170eb8da1b88d3fdfaaf3f7,0.000000
4,b25981de8b5dcdc1c38de111a21312115c34199d,0.000000
5,2e1a61a47232ce57023aeb653ffb9146ee36a819,0.000000
6,418be3d6fef8443de6c8caa171ac82e01784761c,0.000000
7,4d64064dad4c900e4d0dbaf4b327520b1bbe8a4c,0.000000
8,95486443ebebb454403ec8018c555c15fb0b5fbd,0.000000
9,75dc228b5a044c8588a9cf8039273c1a41a73e56,0.052313


In [87]:
location = 'results'
submission.to_csv(F'./{location}/results_6_weighted.csv',index=False)